In [ ]:
import numpy as np
import scipy as sp
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from astropy import units as u
from astropy.coordinates import SkyCoord
from dblinea import DBBase
import widgets  as wg

%reload_ext autoreload
%autoreload 2

In [ ]:
db = DBBase()
schema = 'des_dr2'
table_name = 'main'

In [ ]:
db.get_table_columns(table_name, schema=schema)

In [ ]:
# Define coordinates for the selected region
ra_min, ra_max = 0, 60
dec_min, dec_max = -60, -30

# Query the DES database for objects within the specified region
query_test = f"""
SELECT * FROM {schema}.{tablename}
WHERE ra BETWEEN {ra_min} AND {ra_max}
AND dec BETWEEN {dec_min} AND {dec_max}
"""

query = f"""
SELECT coadd_object_id, ra ,dec, flags_g, mag_auto_g_dered, 
mag_auto_r_dered, mag_auto_i_dered, magerr_auto_g, magerr_auto_r,
magerr_auto_i FROM {schema}.{tablename} WHERE q3c_radial_query(ra, dec, 15.0, -33.7, 0.5) 
AND extended_class_coadd < 2 
"""

# Execute the query and fetch data
data = db.query(query)
data_df = db.fetchall_df(query)

# Save the data to a CSV file
data_df.to_csv("des_data.csv", index=False)


In [ ]:
# Load data
data = pd.read_csv("des_data.csv")

new_columns = {"coadd_object_id": "object_id",
               "mag_auto_g_dered": "mag_g",
               "mag_auto_r_dered": "mag_r",
               "mag_auto_i_dered": "mag_i",
               "magerr_auto_g": "err_g", 
               "magerr_auto_r": "err_r", 
               "magerr_auto_i": "err_i"}

data.rename(columns=new_columns, inplace=True)
dados["gmr"] = dados.mag_g - dados.mag_r

# Distribution of objects
plt.figure(figsize=(10, 6))
plt.hist(data['mag_g'], bins=30, color='skyblue')
plt.xlabel('Magnitude')
plt.ylabel('Frequency')
plt.title('Distribution of Object Magnitudes')
plt.grid(True)
plt.show()

# Color-Magnitude Diagram
plt.figure(figsize=(10, 6))
plt.scatter(data['gmr'], data['mag_g'], alpha=0.5)
plt.gca().invert_yaxis()  # Invert y-axis for magnitude
plt.xlabel('Color Index')
plt.ylabel('Magnitude')
plt.title('Color-Magnitude Diagram')
plt.grid(True)
plt.show()


In [ ]:
wg.corr_analysis(data)

In [ ]:
target = "mag_auto_g_dered"

In [ ]:
features = wg.pc_analysis(data, target) # PCA is not working for strong negative correlations

In [ ]:
X = data[features[0]].values
y = data[target].values

plt.figure(figsize=(10, 6))
plt.scatter(X,y)
plt.grid(True)
plt.show()

In [ ]:
model, y_out = wg.modeling(X.reshape(-1,1), y)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X,
                                                    y,
                                                    test_size=0.3,
                                                    random_state=42)
y_pred = model.predict(X_test.reshape(-1,1))
wg.results_analysis(y_pred, y_test)

In [ ]:
gmag = np.linspace(5, 12, 20)
res = model.predict(gmag.reshape(-1,1))

plt.figure(figsize=(10, 6))
plt.scatter(gmag,res)
plt.grid(True)
plt.show()